# Classical Monte Carlo of $S_N$ Model on Hypertree Lattice

## Statistical Mechanics

### Generic Model
The most general StatMech model we consider is specified by the following ingredients:
* **Lattice**: we will only consider *bipartite* lattice, on which the block Gibbs sampling can be carried out.
 * The bipartite lattice contains two **sublattices** labeled by *A* and *B*.
 * The **boundary**, labeled by *C*, refers to the sites that will not be updated in Monte Carlo. The boundary configuration is fixed by the boundary condition and will not be updated. 
* **Group**: on each site, the degree of freedom is a permutation group element $g_i\in S_n$, which we will called the "spin" hereinafter.
 * Each spin state will be represented as a vector in the canonical representation of $S_n$. For example, for $S_2$ group: $|()\rangle=(1,0)^\intercal$, $|(12)\rangle=(0,1)^\intercal.$
* **Hamiltonian**:
$$H[g] = -\sum_{\langle ij\rangle} K_{ij}\chi(g_i,g_j),$$
where $\chi(g_i,g_j)=\text{Tr}g_i^{-1}g_j$ is the group character (the number of cycles). The partition function is given by
$$Z = -\sum_{[g|A\cup B]}e^{-H[g]} = e^{-F[g|C]}.$$
With given boundary, the free energy $F$ is a function of the boundary configuration.

### Monte Carlo Algorithm
The basic idea of importance sampling is to set up a suitable Markov chain that draws configurations according to the Boltzmann weight
$$P[g] = \frac{e^{-H[g]}}{Z}.$$
There are two classes of update schemes:
* Local update algorithms (Metropolis, **Heat-Bath**)
* Cluster update algorithms (Swendsen-Wang Cluster, Wolff Cluster)

The cluster update greately reduces the dynamic scaling at the critical point, which speed up the convergence in the critical region. Cluster update works well for Potts model, where the spins interacts via delta function: such that the domain wall between all different spins are of the same tension (energy). In the Wolff cluster algorithm, a site is first choosen, and neighboring sites of the same spin can attach to the cluster with probability $p$ (sites of different spins should not attach), and finally the whole cluster is flipped. The transition rate can satisfy the detail balance because the intereial weight is the same ($=p^{C-1}$ where $C$ is number of cluster site, this is because the cluster is grown following a tree, and all trees are equivalent in bond number if the site number is fixed), so the only difference is on the boundary. There are two types of cluster boundaries: boudaries between different spins (of tension 0) and boundary between same spins (of tension $-\ln p$). After flipping, the two types of boundary will switch their positions, so it can match the detail balance condition. More precisely, the partition function for $n$-state Potts model can be cast into
$$Z\propto\sum_{\text{config}}p^{\text{#link}}n^{\text{#cluster}}.$$
This is due to the nice delta function structure. However for our $S_n$ model, the delta interaction is soften in some sense. Then the partition function no longer has the nice link and cluster structure, which make it hard to do cluster update.

While it remains a question of <font color='red'>how to design a cluster update algorithm for $S_n$ model</font>, we will try the local update first, in particular the heat-bath update. The current code uses **block Gibbs sampling** + **heat-bath update**, alternatively sampling on A and B sublattices.

Question: <font color='red'>How to determine equilibrium?</font>

### Temperature Profiling

Question: <font color='red'>How to calculate free energy?</font>

## Code Structure
Objects:
* **Group**
* **Lattice** (SquareLattice, HypertreeLattice)
* **Model** (LatticeModel)
    * Wrapper of the FORTRAN extension MC (provides interface for data transfer and control).

In [1]:
%reload_ext autoreload
%autoreload 2
from MonteCarlo import *

### `Group` Object

**`Group(n)`** represents a permutation group $S_n$.
- Each element is a tuple of $n$ numbers, representing the permutation of $(0,1,...,n-1)$.

#### Attributes
- `Group.dof` gives the order of the group, i.e. $n!$, which is also the degrees of freedom on each site.
- `Group.chi` the table of $\chi(g_1,g_2)$ (adjusted by $-n$).
- `Group.element` gives the list of all group elements.
- `Group.index` is a dictionary that maps group element to its index.

After initialization, `Group.dof` and `Group.chi` should have been set.

#### Methods

In [2]:
'''multiplication table'''
group = Group(3)
[[group.index[group.multiply(g,h)] for h in group] for g in group]

[[0, 1, 2, 3, 4, 5],
 [1, 0, 4, 5, 2, 3],
 [2, 3, 0, 1, 5, 4],
 [3, 2, 5, 4, 0, 1],
 [4, 5, 1, 0, 3, 2],
 [5, 4, 3, 2, 1, 0]]

In [8]:
'''check group inversion'''
[group.index[group.multiply(g, group.inverse(g))] for g in group]

[0, 0, 0, 0, 0, 0]

In [3]:
'''characters'''
{g: group.character(g) for g in group}

{(0, 1, 2): 3,
 (0, 2, 1): 2,
 (1, 0, 2): 2,
 (1, 2, 0): 1,
 (2, 0, 1): 1,
 (2, 1, 0): 2}

The character of a permutation is defined as the number of cycles in the permutation, denoted as $\ln\text{Tr}g$, ranging from $1$ to $n$. Upon initialization, the <code>Group</code> object generates the table of
$$\chi(g_1,g_2)=\ln\text{Tr} g_1 g_2^{-1}-n.$$
One can show that $\chi(g_1,g_2)=\chi(g_2,g_1)$ and $\forall g: \chi(gg_1,gg_2)=\chi(g_1g,g_2g)=\chi(g_1,g_2)$.
* This is the interaction function between two spins across a bond.
* The shift $-n$ is needed to avoid partition weight overflow at low temperature. 

In [4]:
Group(3).chi

[[0, -1, -1, -2, -2, -1],
 [-1, 0, -2, -1, -1, -2],
 [-1, -2, 0, -1, -1, -2],
 [-2, -1, -1, 0, -2, -1],
 [-2, -1, -1, -2, 0, -1],
 [-1, -2, -2, -1, -1, 0]]

### `Lattice` Object

**`Lattice()`** defines a genaric lattice object (bydefault an empty lattice). 

Different types of lattices are realized as **inheritances** of the `Lattice` class:
- `HypertreeLattice(L,Ks,branches)` for hyertree lattice.
- `SquareLattice(L,Ks)` for square lattice.

For the purpose of block-Gibbs update, the lattice should be a **bipartite graph**.
- The **bulk** sites are partitioned to A and B sublattices.
  - The numbers of sites in A and B are denoted as `Lattice.na` and `Lattice.nb`.
  - There is no lattice bond with in the same sublattice.
- The **boundary** sites are ascribed to the C sublattice.
  - The number of boundary sites is `Lattice.nc`
- The total number of sites (including bulk and boundary) is `Lattice.nsite`

**Site Ordering Rules**. The sites are arranged according to the following rules:
- *Bulk sites befor boundary sites*. A, B must come before C.
  - `MC` module because will not allocate state variables for C sites
- *Contineous index* for A and B sites, s.t. within each sublattice dont scatter around.

#### `Site` Subclass

Each inheritance lattice class should contain a definition of the **`Site` subclass** that is specific to the particular type of the hosting lattice (because different lattice may have different coordinate systems and sublattice partition scheme). The `Site` subclass should contain the following
- **Attributes**:
  - `coordinate` a tuple of the coordinate of the site. This attribute is usually set at initialization.
  - `label='A','B','C'` labels the sublattice of the site. This will be set by the method `get_label(lattice)`.
  - `index` keeps the site index in the lattice, will be set by the lattice initialization.
  - One may also introduce attributes for each coordinate component (but that is not required).
- **Methods**:
  - `get_label(lattice)` set the attribute `label` to: `A`,`B` or `C`, and return it. It takes the hosting lattice as its input, such that the global parameters of the lattice can be passed with the entire lattice object.
  - `sortkey()` returns the key for sorting the site.

#### Attributes

There are two groups of attributes:
- Lattice structural attributes
  - `nsite`, `na`, `nb`, `nc`. They must satisfy `na + nb + nc = nsite`.
  - `sites` (site container) is a list of all sites in the lattice (including both bulk and boundary).
  - `coordmap` is a dictionary that maps the coordinates to the site index.
- Hamiltonian related attributes
  - `irng` a list of length `nsite+1` to specify the partition of `jlst` and `klst`, s.t. the data block for site `i` is specified by the slice `irng[i]:irng[i+1]`.
  - `jlst`, `klst` lists of j-index and corresponding coupling strength K. Both lists are of the length `nlst`.
  
#### Methods

Each inheritance lattice clase must redefine the follwing methods:
- `get_site()` return a list of all lattice sites (no need to be labeled, sorted or indexed, just create objects).
- `Hamiltonian()` assuming the *lattice has been assembled* (sites labeled, sorted and indexed), return the Hamiltonian terms as `(i,j,K)` tuples, such that $H=-\sum_{i,j}K_{ij}\chi(g_i,g_j)$

#### Hypertree Lattice
- **`HypertreeLattice(L, Ks, branches)`**: creates a hypertree lattice of
  - `L` sites in the ultimate UV layer,
  - Make specific number of `branches` (default = 2) towards IR (which is also the number of sites in each block).
    - For each given depth and feature, the spacial sites are partitions into groups of sizes upto `branches`.
    - Each group develops new features in the deeper IR layer, and the connection is *bipartite complete*.
    - If a group contains only one site, it will not develop new IR features (lattice will terminate in this way).
  - The coupling strengths are provided by `Ks`, as list or as array.
    - `Ks=[K0,K1,K2,...]` are the coupling strengths to each branches.
    - If `Ks` contains *less* couplings then the number of `branches`, unspecified branches will be *decoupled*.
    - If `Ks` contains *more* couplings than the number of `branches`, extra couplings will be *neglected*.
- Sublattice partition convension:
  - A sublattice must contain the **ultimate UV** sites, which are also arranged to the **front**, such that the measurement of these sites can be conviniently specified.
  - C sublattice are the **ultimate IR** site, which are arranged to the **back**.

In [2]:
lattice = HypertreeLattice(6,Ks=[1.,2.,3.],branches=3)
for key in ('nsite','na','nb','nc','nlst','irng','jlst','klst'):
    print(key,':',vars(lattice)[key])
for site in lattice:
    print('%s%s - IR: %s UV: %s'%(site.label,repr(site),repr(site.IR),repr(site.UV)))

nsite : 18
na : 6
nb : 6
nc : 6
nlst : 60
irng : [ 1  3  5  7  9 11 13 18 23 28 33 38 43 46 49 52 55 58 61]
jlst : [ 7  8  7  8  9 10  9 10 11 12 11 12  1  2 13 14 15  1  2 16 17 18  3  4 13
 14 15  3  4 16 17 18  5  6 13 14 15  5  6 16 17 18  7  9 11  7  9 11  7  9
 11  8 10 12  8 10 12  8 10 12]
klst : [ 1.  1.  2.  2.  1.  1.  2.  2.  1.  1.  2.  2.  1.  2.  1.  1.  1.  1.
  2.  1.  1.  1.  1.  2.  2.  2.  2.  1.  2.  2.  2.  2.  1.  2.  3.  3.
  3.  1.  2.  3.  3.  3.  1.  2.  3.  1.  2.  3.  1.  2.  3.  1.  2.  3.
  1.  2.  3.  1.  2.  3.]
A0:<2,0,0> - IR: [] UV: [6:<1,0,0>, 7:<1,0,1>]
A1:<2,1,0> - IR: [] UV: [6:<1,0,0>, 7:<1,0,1>]
A2:<2,2,0> - IR: [] UV: [8:<1,1,0>, 9:<1,1,1>]
A3:<2,3,0> - IR: [] UV: [8:<1,1,0>, 9:<1,1,1>]
A4:<2,4,0> - IR: [] UV: [10:<1,2,0>, 11:<1,2,1>]
A5:<2,5,0> - IR: [] UV: [10:<1,2,0>, 11:<1,2,1>]
B6:<1,0,0> - IR: [0:<2,0,0>, 1:<2,1,0>] UV: [12:<0,0,0>, 13:<0,0,1>, 14:<0,0,2>]
B7:<1,0,1> - IR: [0:<2,0,0>, 1:<2,1,0>] UV: [15:<0,0,3>, 16:<0,0,4>, 17:<0,0,5>]
B

#### Square Lattice
- **`SquareLattice(L, Ks)`**: creates a square lattice of
  - `L` x `L` sites with *periodic* boundary condition along both dimensions.
  - There is only one coupling, uniform through out the lattice, given by `K[0]`.
    - If `Ks` contains more than one couplings, extra couplings will be *neglected*.
    - But it is possible that in future, extra couplings in `Ks` can be assigned to 2nd, 3rd neighbors ect.
- Sublattice partition convension:
  - A = even sublattice, B = odd sublattice. Such a partition requies: **`L` must be even**!
  - There is no boundary sites. So C is empty.

In [5]:
#%run 'MonteCarlo.py'
lattice = SquareLattice(4,[1.])
for key in ('nsite','na','nb','nc','nlst','irng','jlst','klst'):
    print(key,':',vars(lattice)[key])
for site in lattice:
    print('%s%s - neighbors: %s'%(site.label,repr(site),repr(site.neighbors)))

nsite : 16
na : 8
nb : 8
nc : 0
nlst : 64
irng : [ 1  5  9 13 17 21 25 29 33 37 41 45 49 53 57 61 65]
jlst : [ 9 10 11 15  9 10 12 16  9 11 12 13 10 11 12 14 11 13 14 15 12 13 14 16  9
 13 15 16 10 14 15 16  1  2  3  7  1  2  4  8  1  3  4  5  2  3  4  6  3  5
  6  7  4  5  6  8  1  5  7  8  2  6  7  8]
klst : [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
A0<0,0> - neighbors: [[14<3,0>, 9<0,3>, 8<0,1>, 10<1,0>]]
A1<0,2> - neighbors: [[15<3,2>, 8<0,1>, 9<0,3>, 11<1,2>]]
A2<1,1> - neighbors: [[8<0,1>, 10<1,0>, 11<1,2>, 12<2,1>]]
A3<1,3> - neighbors: [[9<0,3>, 11<1,2>, 10<1,0>, 13<2,3>]]
A4<2,0> - neighbors: [[10<1,0>, 13<2,3>, 12<2,1>, 14<3,0>]]
A5<2,2> - neighbors: [[11<1,2>, 12<2,1>, 13<2,3>, 15<3,2>]]
A6<3,1> - neighbors: [[12<2,1>, 14<3,0>, 15<3,2>, 8<0,1>]]
A7<3,3> - neig

### `Model` Object

The CPU intensive Monte Carlo samplings are performed by the FORTRAN extension MC. The module MC contains two parts:
* core (for MC sampling and update macroscopic observibles)
* physics (for data collection and measurements)

The `Model` object is a *python wapper* for the `MC` module. Because all `Model` instances shares the same `MC` kernel, so **there must only one active `Model` object at any time**. More than two active `Model` objects will interefer with each other.

In [5]:
print(MC.__doc__)

This module 'MC' is auto-generated with f2py (version:2).
Functions:
Fortran 90/95 modules:
  core --- nc,nlst,nsite,chi,irng,nb,dof,klst,energy,config,na,jlst,hist,init(),run(),get_energy(),get_hist(),dump(),load()  physics --- magnet2,nspin,monitor,energy1,energy2,spins,magnet1,measure().


#### Attributes

The three-block data structure: system, state and data.
- **`system`**: static, not changed during MC update, defines the system. 
  - The full set of system parameters are specified by the Model class attribute `Model.system_parameters`
  - During the temperature ramping, only `klst` will change. The other parameters are fixed by the lattice graph and on-site group.
- **`state`**: dynamic, updated by MC sampling.
  - Can be kept for future restart of the Markov chain.
- **`data`**: disposible, generated by the `measure` method.

Mapping the attributes between **MC** (**FORTRAN** module) and **Model** (**python** object):
- The system and state parameters are linked (as inout) between MC and Model.
- The data parameters are output from MC to Model after each measurement.


    MC                                                        Model
    +- MC.core                                   Model.system[:] -+
    |  +- MC.core.dof                   <==> Model.dof        -+  |
    |  +- MC.core.nsite                 <==> Model.nsite      -+  |
    |  +- MC.core.na                    <==> Model.na         -+  |
    |  +- MC.core.nb                    <==> Model.nb         -+  |
    |  +- MC.core.nc                    <==> Model.nc         -+  |
    |  +- MC.core.nlst                  <==> Model.nlst       -+  |
    |  +- MC.core.chi[:dof,:dof]        <==> Model.chi        -+  |
    |  +- MC.core.irng[:nsite+1]        <==> Model.irng       -+  |
    |  +- MC.core.jlst[:nlst]           <==> Model.jlst       -+  |
    |  +- MC.core.klst[:nlst]           <==> Model.klst       -+  |
    |  |                                          Model.state[:] -+
    |  +- MC.core.config[:nsite]        <==> Model.config     -+  |
    |  +- MC.core.energy                <==> Model.energy     -+  |
    |  +- MC.core.hist[:dof]            <==> Model.hist       -+  |
    +- MC.physics                                                 |
       +- MC.physics.nspin                                        |
       +- MC.physics.monitor[:nspin]               Model.data[:] -+
       +- MC.physics.energy1            ===> Model.data[energy1]
       +- MC.physics.energy2            ===> Model.data[energy2]
       +- MC.physics.magnet1[:dof]      ===> Model.data[magnet1]
       +- MC.physics.magnet2[:dof,:dof] ===> Model.data[magnet2]
       +- MC.physics.spins[:dof,:nspin] ===> Model.data[spins]
                                             Model.data[steps]       

Notes:
- The `Model` attributes *do not physically exist*. They are accessed by `getattr` and `setattr` methods. The system and state attributes can be set as a whole by passing the dictionary. The data attributs are read only, setting `Model.data` will not affect the variables in the MC module.
- `Model` object provides **state parameter parsing**.
  - `config='FM','uniform'`: initialize with 0's; `'PM','random'`: initialize with random spins from `range(dof)`.
  - `energy='unknown','nan'`: set `energy` to nan (which is a flag telling MC that the energy has not be calculated yet).
  - `hist='unknown'`: set `hist` to empty with `hist[0]=-1` (which is a flag telling MC that the histogram has not been collected yet).
- When attribute getter is called to get `energy` and `hist`, it is first checked whether they have been calculated, if not, will evoke MC module (by calling `MC.core.get_energy()` or `.get_hist()`) to calculate and then return.
- `MC.core.dump()` and `MC.core.load()` can be called to dump/load core variables.

#### Methods

`Model.run(steps, mode)` runs the MC updates for specific `steps`, under the `mode`:
- `mode = 0` (default): update without keeping track of the `energy` and `hist`.
- `mode = 1`: `energy` and `hist` are updated with the MC run at every spin flip.

`Model.measure(step, monitor)` runs MC updates for specific `steps`, and returns the measurement on the samples of:
- energy 1st and 2nd moment: $\langle E\rangle$, $\langle E^2\rangle$
- total magnetization 1st and 2nd moment: $\langle M\rangle$, $\langle M^2\rangle$ (where $M=N_{A\cup B}^{-1}\sum_{i\in A\cup B} g_i$ and $g_i$ is represented as a vector in the canonical representation).
  - the 2nd moment $\langle M^2\rangle$ is a matrix, containing infomations of cross correlation.
- specific spins (averged over samples), spin indices are specified by <code>monitor</code>, and the number of spin to monitor is specified by <code>nspin</code>.
- the number of steps (samples) is also returned, such that the data can be combined with previous measurements properly, as weighted by the number of samples.

#### Core Performance

**Heat-Bath** update: on each site $i$, randomly draw a new spin state $g_i$ (regardless of the previous spin state) with probability:
$$p(g_i)=\frac{e^{h(g_i)}}{\sum_{g'_i}e^{h(g'_i)}}, h(g_i)=\sum_{j \text{n.n.}i}K_{ij}\chi(g_i,g_j).$$

**Block-Gibbs** sampling: spins in the same sublattice are block-updated.
- For A sublattice: `I=1:NA`
- For B sublattice: `I=NA+1:NA+NB`.

FROTRAN implementation (for each spin update):
- `RAND_NUMBER`: initialize `RND` by a random real drawn from the uniform distribution in $[0,1]$. gfortran implements KISS random number generator, $2^{123}$.
- `SET_BLOCK`: sets the cumulated weights and renoralized random real
  - `SET_BIAS`: calculate the on-site bias fields (site indexed by `I`) as `sum(CHI[:,CONFIG[JLST[J]]]*KLST[J] for J in range(IRNG[I],IRNG[I+1]))`. The result is a vector of `DOF` components, which specify the bias field for all group elements.
  - `SET_WEIGHT`: exponentiate the bias fields and accumulate to unormalized CDF weights. The time of this step can vary from 4ns to 10ns depending on the value of the exponent. `EXP(x)` has a quick return for `x=0` but slower for generic `x`. The last compunent is the total partition weight.
  - `RND *= WEIGHT[-1]`: the random real is then multiplied by the total partition weight, such that it is distributed uniformly from 0 to the total weight.
- `CHOOSE`: `WEIGHT` and `RND` is then used to make choice of the new spin state, implemented by a binary serach to determine which bin in `WEIGHT` does `RND` fall into.

In each step (sweep) of MC sampling, the *entire bulk is updated* (which contains two block Gibbs sampling: A-given-B and B-given-A). Measurements are made after the bulk update, so the measurement time is neglegable. For one MC step, the complexity depends on:
- $N$ - the number of bulk sites.
- $n!$ - group order (onsite degrees of freedom).

Total time estimation: $N(12 n! + 6\log(n!) + 16)$ ns (CPU: 3GHz Intel Core i7)

<table align="left">
<tr><td>procedure</td><td>complexity</td><td>time (ns)</td></tr>
<tr><td><code>RAND_NUMBER</code></td><td>$N$</td><td>13.8</td></tr>
<tr><td><code>SET_BLOCK</code></td><td>~$Nn!$</td><td>12.1</td></tr>
<tr><td><code>    SET_BIAS</code></td><td>$Nn!$</td><td>2.3</td></tr>
<tr><td><code>    SET_WEIGHT</code></td><td>$Nn!$</td><td>9.8</td></tr>
<tr><td><code>        EXP</code></td><td>$Nn!$</td><td>8.0</td></tr>
<tr><td><code>    RND</code></td><td>$N$</td><td>2.1</td></tr>
<tr><td><code>CHOOSE</code></td><td>$N\log(n!)$</td><td>5.6</td></tr>
</table>

Average update time per spin scales with $n$ as:
<table align="left">
<tr><td>$n=$</td><td>2</td><td>3</td><td>4</td><td>5</td><td>6</td></tr>
<tr><td>time(μs)</td><td>0.056</td><td>0.12</td><td>0.35</td><td>1.52</td><td>9.06</td></tr>
</table>

**Profiling** code:

In [40]:
%reload_ext snakeviz
%snakeviz LatticeModel(HypertreeLattice(128,[0.75,0.25]),Group(2)).run(10000,0)

 
*** Profile stats marshalled to file '/var/folders/lt/kgph6qbj6y306pfhbyd1kn2r0000gn/T/tmpvg_g9b7o'. 


#### Model Construction

- **`Model(system,state,data)`** creates a wrapper of the `MC` module.
  - `system` argument is required, passed in as a dictionary, which must contains all the keys specified in the class variable `Model.system_parameters` (`ValueError` will be raised if any key missing).
  - `state` and `data` arguments are optional.
  - This construction is useful to reload the model and continue the simulation from saved states.
- **`LattticeModel(lattice,group)`** creates a initialized model from `lattice` and `group`, as an inheritance of `Model`.
  - `lattice` is a `Lattice` object and `group` is a `Group` object. They provide the necessary data to build a model.

In [17]:
LatticeModel(HypertreeLattice(4,[1.,0.2]),Group(2))

## Hypertree Model

### Example
Generate model, run, get hist and energy, make measurements.
- use **block Gibbs sampling** + **heatbath update**, alternatively sampling on A and B sublattices.
  - A sublattice is the one that contains the most UV layer
  - B sublattice contains the rest
  - the boundary is labeled by C
- energy is calculated for all sites including the boundary spins.
- hist and magnetization only conts the bulk spins, boundary spins are not taken into account.

In [18]:
mdl = LatticeModel(HypertreeLattice(4,[1.,0.2]),Group(3)).run(100)
for k in range(10):
    mdl.run(mode = 1)
    print(mdl.config,mdl.hist,mdl.energy)
mdl.measure(1000,monitor=[0,1,2,3])

[5 5 5 3 5 0 5 5 0 0 0 0] [1 0 0 1 0 6] 10.799999999999999
[0 5 5 4 1 0 2 5 0 0 0 0] [2 1 1 0 1 3] 13.2
[0 3 4 0 0 0 1 5 0 0 0 0] [4 1 0 1 1 1] 10.4
[0 3 4 0 0 0 1 4 0 0 0 0] [4 1 0 1 2 0] 10.0
[0 4 1 1 0 0 1 3 0 0 0 0] [3 3 0 1 1 0] 9.6
[0 2 2 0 2 0 2 2 0 0 0 0] [3 0 5 0 0 0] 10.8
[0 4 2 1 0 0 2 0 0 0 0 0] [4 1 2 0 1 0] 9.200000000000001
[1 3 2 5 0 0 2 4 0 0 0 0] [2 1 2 1 1 1] 12.000000000000002
[0 5 2 3 0 0 1 3 0 0 0 0] [3 1 1 2 0 1] 11.200000000000003
[0 3 1 4 0 0 1 1 0 0 0 0] [3 3 0 1 1 0] 8.400000000000004


{'energy1': 11.225199999999987,
 'energy2': 131.27855999999952,
 'magnet1': array([ 0.419125,  0.137375,  0.14325 ,  0.084125,  0.084625,  0.1315  ]),
 'magnet2': array([[ 0.20951563,  0.04964063,  0.05325   ,  0.02954688,  0.02940625,
          0.04776562],
        [ 0.04964063,  0.03904688,  0.01346875,  0.01059375,  0.01032813,
          0.01429688],
        [ 0.05325   ,  0.01346875,  0.0421875 ,  0.01125   ,  0.00960937,
          0.01348437],
        [ 0.02954688,  0.01059375,  0.01125   ,  0.01842188,  0.00528125,
          0.00903125],
        [ 0.02940625,  0.01032813,  0.00960937,  0.00528125,  0.01995312,
          0.01004688],
        [ 0.04776562,  0.01429688,  0.01348437,  0.00903125,  0.01004688,
          0.036875  ]]),
 'spins': array([[ 0.576,  0.247,  0.235,  0.188],
        [ 0.128,  0.158,  0.158,  0.176],
        [ 0.117,  0.165,  0.184,  0.178],
        [ 0.032,  0.123,  0.125,  0.146],
        [ 0.039,  0.118,  0.142,  0.149],
        [ 0.108,  0.189,  0.156,  0